In [1]:
from __future__ import print_function
#from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle
import tensorflow as tf
from tensorflow.python.ops import rnn_cell, rnn

In [2]:
#load data first
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS+NWP.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    test_dataset = save['test_dataset']
    train_old = save['t_v_dataset']
    del save


test_data_1 = test_dataset['data_ASOS']
test_data_2 = test_dataset['data_NWP']
test_label = test_dataset['label']
train_data_1 = train_old['data_ASOS']
train_data_2 = train_old['data_NWP']
train_label = train_old['label']

In [3]:
#dataset normalize
mean1 = train_data_1.mean(axis = 0)
std1 = train_data_1.std(axis = 0)
print(mean1.shape, std1.shape)
train_data_1_n = (train_data_1 - mean1)/std1
#validate_data_n = (validate_data - mean)/std
test_data_1_n = (test_data_1 - mean1)/std1

(70,) (70,)


In [4]:
#dataset normalize
mean2 = train_data_2.mean(axis = 0)
std2 = train_data_2.std(axis = 0)
print(mean2.shape, std2.shape)
train_data_2_n = (train_data_2 - mean2)/std2
#validate_data_n = (validate_data - mean)/std
test_data_2_n = (test_data_2 - mean2)/std2

(12,) (12,)


In [5]:
train_data = np.hstack((train_data_1_n[:, :63], train_data_1_n[:, -1:],train_data_1_n[:, 63:-1] ))
test_data = np.hstack((test_data_1_n[:, :63], test_data_1_n[:, -1:],test_data_1_n[:, 63:-1] ))

In [6]:
train_data = np.concatenate((train_data.reshape(train_data.shape[0], 7, 10), 
                             np.repeat(train_data_2_n, 7, axis = 0).reshape(train_data_2_n.shape[0], 7, 12)), axis = 2)
test_data = np.concatenate((test_data.reshape(test_data.shape[0], 7, 10), 
                             np.repeat(test_data_2_n, 7, axis = 0).reshape(test_data_2_n.shape[0], 7, 12)), axis = 2)

In [7]:
train_data.shape, test_data.shape

((116657, 7, 22), (29165, 7, 22))

In [8]:
train_data.shape, test_data.shape

((116657, 7, 22), (29165, 7, 22))

In [9]:
train_data_2d = train_data.reshape(train_data.shape[0], 7, 22, 1)
test_data_2d = test_data.reshape(test_data.shape[0], 7, 22, 1)
train_label = train_label.reshape(-1, 1)
test_label = test_label.reshape(-1, 1)

In [10]:
train_data_2d.shape, train_label.shape

((116657, 7, 22, 1), (116657, 1))

regression problems

In [ ]:
batch_size = 128
patch_size = 3
depth = 16
num_hidden = 64
num_channels = 1
num_labels = 1
num_feature = 22

In [ ]:
graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, 7, num_feature, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_test_dataset = tf.constant(test_data_2d, dtype=tf.float32)
    tf_test_labels = tf.constant(test_label, dtype=tf.float32) 
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([num_feature * 7 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    pred = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.abs(pred - tf_train_labels))
    
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.01
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 2000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    MAE = tf.reduce_mean(tf.abs(model(tf_test_dataset) - tf_test_labels))
    

In [ ]:
num_steps = 100001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_label.shape[0] - batch_size)
        batch_data = train_data_2d[offset:(offset + batch_size), :, :, :]
        batch_labels = train_label[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, lr = session.run([optimizer, loss, learning_rate], feed_dict=feed_dict)
        if step % 500 == 0:
            print('Minibatch loss at step %d learning rate = %f: loss = %f, MAE = %f' % (step, lr, l, MAE.eval()))
            
    

The other type of method to use the NWP model

In [67]:
train_data = np.hstack((train_data_1_n[:, :63], train_data_1_n[:, -1:],train_data_1_n[:, 63:-1] ))
test_data = np.hstack((test_data_1_n[:, :63], test_data_1_n[:, -1:],test_data_1_n[:, 63:-1] ))

In [68]:
train_data.shape, test_data.shape

((116657, 70), (29165, 70))

In [69]:
train_data_2d = train_data.reshape(train_data.shape[0], 7, 10, 1)
test_data_2d = test_data.reshape(test_data.shape[0], 7, 10, 1)
train_label = train_label.reshape(-1, 1)
test_label = test_label.reshape(-1, 1)

In [70]:
train_data_NWP = train_data_2_n
test_data_NWP = test_data_2_n

In [71]:
test_data_NWP.shape, test_data_2d.shape

((29165, 12), (29165, 7, 10, 1))

In [80]:
batch_size = 128
patch_size = 7
depth = 32
num_hidden = 64
num_channels = 1
num_labels = 1
num_feature = 10

In [84]:
graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset1 = tf.placeholder(tf.float32, shape=(batch_size, 7, num_feature, num_channels))
    tf_train_dataset2 = tf.placeholder(tf.float32, shape=(batch_size, 12))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_test_dataset1 = tf.constant(test_data_2d, dtype=tf.float32)
    tf_test_dataset2 = tf.constant(test_data_NWP, dtype=tf.float32)
    tf_test_labels = tf.constant(test_label, dtype=tf.float32) 
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, 1, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, 1, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([num_feature * 7 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden + 12, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data1, data2):
        conv = tf.nn.conv2d(data1, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        concat = tf.concat(1, [hidden, data2])
        return tf.matmul(concat, layer4_weights) + layer4_biases
  
    # Training computation.
    pred = model(tf_train_dataset1, tf_train_dataset2)
    loss = tf.reduce_mean(tf.abs(pred - tf_train_labels))
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.02
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 4000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    MAE = tf.reduce_mean(tf.abs(model(tf_test_dataset1, tf_test_dataset2) - tf_test_labels))

In [85]:
st = time.time()
num_steps = 200001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_label.shape[0] - batch_size)
        batch_data1 = train_data_2d[offset:(offset + batch_size), :, :, :]
        batch_data2 = train_data_NWP[offset:(offset + batch_size), :]
        batch_labels = train_label[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset1 : batch_data1,  tf_train_dataset2 : batch_data2, tf_train_labels : batch_labels}
        _, l, lr = session.run([optimizer, loss, learning_rate], feed_dict=feed_dict)
        if step % 500 == 0:
            print('Minibatch loss at step %d learning rate = %f: loss = %f, MAE = %f' % (step, lr, l, MAE.eval()))
et = time.time()
print(et - st)

Initialized
Minibatch loss at step 0 learning rate = 0.020000: loss = 9.519822, MAE = 14.396765
Minibatch loss at step 500 learning rate = 0.020000: loss = 1.250537, MAE = 0.684987
Minibatch loss at step 1000 learning rate = 0.020000: loss = 0.841218, MAE = 0.633878
Minibatch loss at step 1500 learning rate = 0.020000: loss = 0.814739, MAE = 0.839496
Minibatch loss at step 2000 learning rate = 0.020000: loss = 0.515709, MAE = 0.597903
Minibatch loss at step 2500 learning rate = 0.020000: loss = 0.582989, MAE = 0.684882
Minibatch loss at step 3000 learning rate = 0.020000: loss = 0.999996, MAE = 0.603188
Minibatch loss at step 3500 learning rate = 0.020000: loss = 0.960646, MAE = 0.567667
Minibatch loss at step 4000 learning rate = 0.019000: loss = 0.536711, MAE = 0.591064
Minibatch loss at step 4500 learning rate = 0.019000: loss = 0.349664, MAE = 0.567524
Minibatch loss at step 5000 learning rate = 0.019000: loss = 0.472264, MAE = 0.454232
Minibatch loss at step 5500 learning rate = 0

Classification problems

In [63]:
from __future__ import division
import time

In [37]:
from sklearn.preprocessing import Binarizer
pre = Binarizer(threshold = 1.01)
b_train_label = pre.transform(train_label.reshape(1, -1))
b_test_label = pre.transform(test_label.reshape(1, -1))

In [38]:
c_train_label = 1 - b_train_label[0]
c_test_label = 1 - b_test_label[0]

In [39]:
train_data_n = np.hstack((train_data_1_n, train_data_2_n))

In [40]:
train_data_n_resample = train_data_n
c_train_label_resample = c_train_label

In [41]:
#need to balanced the dataset
from unbalanced_dataset.over_sampling import SMOTE
sm = SMOTE(ratio = 0.03, kind='regular')
train_data_n_resample, c_train_label_resample = sm.fit_transform(train_data_n, c_train_label)

Determining classes statistics... 2 classes detected: Counter({0.0: 114677, 1.0: 1980})
Finding the 5 nearest neighbours...
done!
Creating synthetic samples...Generated 1460 new samples ...
done!


In [42]:
train_data_n_resample_ASOS = train_data_n_resample[:, :70]
train_data_n_resample_NWP = train_data_n_resample[:, 70:]

In [43]:
train_data_ASOS_1d = np.hstack((train_data_n_resample_ASOS[:, :63], train_data_n_resample_ASOS[:, -1:],train_data_n_resample_ASOS[:, 63:-1] ))
train_data_ASOS_2d = train_data_ASOS_1d.reshape(train_data_ASOS_1d.shape[0], 7, 10, 1)

In [44]:
c_train_label[:10]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [45]:
# change from Indice to Vector
''''''
def makeIndicatorVars(T):
    # Make sure T is two-dimensiona. Should be nSamples x 1.
    if T.ndim == 1:
        T = T.reshape((-1,1))    
    return (T == np.unique(T)).astype(int)

In [46]:
v_train_label = makeIndicatorVars(c_train_label.reshape(-1, 1))
v_test_label = makeIndicatorVars(c_test_label.reshape(-1, 1))

In [47]:
v_train_label_resample = makeIndicatorVars(c_train_label_resample.reshape(-1, 1))

In [48]:
v_train_label.shape, v_test_label.shape

((116657, 2), (29165, 2))

In [49]:
#shuffle the data set
arr = np.arange(c_train_label_resample.shape[0])
np.random.shuffle(arr)
train_data_ASOS_2d_s =  train_data_ASOS_2d[arr]
train_data_n_resample_NWP_s = train_data_n_resample_NWP[arr]
v_train_label_resample_s = v_train_label_resample[arr]

In [60]:
batch_size = 128
patch_size = 7
depth = 16
num_hidden = 64
num_channels = 1
num_labels = 2
num_feature = 10
ratio = 0.1

In [61]:
graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset1 = tf.placeholder(tf.float32, shape=(batch_size, 7, num_feature, num_channels))
    tf_train_dataset2 = tf.placeholder(tf.float32, shape=(batch_size, 12))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_test_dataset1 = tf.constant(test_data_2d, dtype=tf.float32)
    tf_test_dataset2 = tf.constant(test_data_NWP, dtype=tf.float32)
    tf_test_labels = tf.constant(v_test_label, dtype=tf.float32) 
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, 1, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, 1, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([num_feature * 7 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden + 12, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data1, data2):
        conv = tf.nn.conv2d(data1, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        concat = tf.concat(1, [hidden, data2])
        return tf.matmul(concat, layer4_weights) + layer4_biases

    def acc(predict, label):
        #correct_prediction = tf.equal(predicted_label, tf_train_label)
        correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(label, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        predict_event = tf.reduce_sum(tf.argmax(predict, 1))
        label_event = tf.reduce_sum(tf.argmax(label, 1))
        true_positive = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 2), tf.int64))
        true_negative = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 0), tf.int64))
        false_positive = predict_event - true_positive 
        false_negative = label_event - true_positive
        return accuracy, false_positive, false_negative, true_positive, true_negative
    def ROC(FP, FN, TP, TN):
        TP_percent = TP / (TP + FN) 
        FP_percent = FP / (FP + TN) 
        return TP_percent, FP_percent
    
    def PRC(FP, FN, TP, TN):
        precision = TP / (TP + FP + 1)
        recall = TP / (TP + FN + 1)
        csi = TP / (TP + FP + FN)
        return precision, recall, csi    
    
    # Training computation.
     # Training computation.
    pred = model(tf_train_dataset1, tf_train_dataset2)
    class_weight = tf.constant([ratio, 1.0 - ratio])
    weighted_pred = tf.mul(pred, class_weight) # shape [batch_size, 2]
    #loss = tf.reduce_mean(tf.abs(pred - tf_train_labels))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(weighted_pred, tf_train_labels)) # Softmax loss
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.03
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 2000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    
    
    # Evaluate model
    test_pred = model(tf_test_dataset1, tf_test_dataset2)
    correct_pred = tf.equal(tf.argmax(test_pred,1), tf.argmax(tf_test_labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    test_acc, FP, FN, TP, TN = acc(test_pred, tf_test_labels)
    pre, rec, csi = PRC(FP, FN, TP, TN)
    
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(csi, global_step = global_step)

In [64]:
st = time.time()
num_steps = 40001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_label.shape[0] - batch_size)
        batch_data1 = train_data_ASOS_2d_s[offset:(offset + batch_size), :, :, :]
        batch_data2 = train_data_n_resample_NWP_s[offset:(offset + batch_size), :]
        batch_labels = v_train_label_resample_s[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset1 : batch_data1,  tf_train_dataset2 : batch_data2, tf_train_labels : batch_labels}
        _, l, lr = session.run([optimizer, loss, learning_rate], feed_dict=feed_dict)
        if step % 500 == 0:
            print('Minibatch loss at step %d learning_rate = %f: loss = %f' % (step, lr, l))
            tp, fp, fn, tn, precision, recall, f_score= session.run([TP, FP, FN, TN, pre, rec,csi])
            print("TP = %d, FP = %d, FN = %d, TN = %d" % (tp, fp, fn, tn))
            print("precision = %f, recall = %f, csi = %f" % (precision, recall, f_score))
et = time.time()
print(et-st)

Initialized
Minibatch loss at step 0 learning_rate = 0.030000: loss = 0.206536
TP = 0, FP = 0, FN = 421, TN = 28744
precision = 0.000000, recall = 0.000000, csi = 0.000000
Minibatch loss at step 500 learning_rate = 0.030000: loss = 0.029069
TP = 293, FP = 315, FN = 128, TN = 28429
precision = 0.481117, recall = 0.694313, csi = 0.398098
Minibatch loss at step 1000 learning_rate = 0.030000: loss = 0.127750
TP = 308, FP = 404, FN = 113, TN = 28340
precision = 0.431978, recall = 0.729858, csi = 0.373333
Minibatch loss at step 1500 learning_rate = 0.030000: loss = 0.065384
TP = 284, FP = 236, FN = 137, TN = 28508
precision = 0.545106, recall = 0.672986, csi = 0.432268
Minibatch loss at step 2000 learning_rate = 0.028500: loss = 0.054908
TP = 310, FP = 354, FN = 111, TN = 28390
precision = 0.466165, recall = 0.734597, csi = 0.400000
Minibatch loss at step 2500 learning_rate = 0.028500: loss = 0.034386
TP = 312, FP = 339, FN = 109, TN = 28405
precision = 0.478528, recall = 0.739336, csi = 0.4